In [1]:
import pandas as pd
import numpy as np
import glob, sys, os, time , itertools , warnings
from bs4 import BeautifulSoup
from tqdm import tqdm
import multiprocessing as mp
# from DataProcessing import 

warnings.filterwarnings('ignore')

with open('nogit\\path','r') as f:
    path_to_csv = f.read()

In [2]:
state_codes = pd.read_excel(path_to_csv+'Us-zipcodes\\statecode.xlsx',index_col=0)

In [3]:
#Unemployment

path_to_unemp = path_to_csv+'US-Unemployement\\us-county-unemployement20102018.xlsx'

unempdf = pd.concat([pd.read_excel(path_to_unemp,sheet_name=i,index_col=0) for i in range(9)],axis=1)

col_to_filt = [col for col in unempdf.columns if 'Pct' in col]

unempdf= unempdf[col_to_filt]
unempdf.columns = ['Unemp-'+col for col in unempdf.columns]

unempdf.index = [', '.join([x.strip() for x in ind.replace(' County','').replace('/city','').replace('City','city').split(',')]
                            ) 
                 for ind in unempdf.index]
unempdf = unempdf.rename(index={'District of Columbia':'District of Columbia, DC'})



unempdf.to_csv('csv/unemp.csv')
unempdf.sort_index()

,Unemp-Pct-2014,Unemp-Pct-2015,Unemp-Pct-2016,Unemp-Pct-2017,Unemp-Pct-2018,Unemp-Pct-2013,Unemp-Pct-2012,Unemp-Pct-2011,Unemp-Pct-2010
"Abbeville, SC",7.7,6.9,5.5,4.6,4.0,9.5,10.6,12.5,13.6
"Acadia Parish, LA",5.8,6.8,7.4,5.9,5.4,6.0,6.0,6.7,7.3
"Accomack, VA",6.7,5.4,4.9,4.5,3.8,7.2,7.9,8.4,7.9
"Ada, ID",4.1,3.7,3.3,2.8,2.5,5.2,6.2,7.3,8.3
"Adair, IA",4.0,3.2,2.9,2.8,2.4,4.2,4.4,4.9,5.5
...,...,...,...,...,...,...,...,...,...
"Yuma, AZ",22.9,21.7,18.9,17.0,17.0,25.5,23.9,24.2,25.1
"Yuma, CO",3.3,2.6,2.1,1.5,2.0,4.8,5.1,5.3,5.6
"Zapata, TX",5.2,7.6,10.8,7.4,5.6,6.2,6.3,8.1,10.4
"Zavala, TX",12.1,11.2,13.9,11.2,9.5,14.5,13.5,14.1,13.9


In [4]:
#CPI Data
path_to_cpi = path_to_csv+'US-CPI\\SeriesReport-20191118124049_549fff.xlsx'
cpi_df = pd.read_excel(path_to_cpi,skiprows=11,index_col=0)['Annual']; cpi_df

Year
1990    130.700
1991    136.200
1992    140.300
1993    144.500
1994    148.200
1995    152.400
1996    156.900
1997    160.500
1998    163.000
1999    166.600
2000    172.200
2001    177.100
2002    179.900
2003    184.000
2004    188.900
2005    195.300
2006    201.600
2007    207.342
2008    215.303
2009    214.537
2010    218.056
2011    224.939
2012    229.594
2013    232.957
2014    236.736
2015    237.017
2016    240.007
2017    245.120
2018    251.107
2019    255.657
Name: Annual, dtype: float64

In [5]:
#Income

path_to_income = path_to_csv+'US-Personal-Income\\CAINC1__ALL_STATES_1969_2017.xlsx'
income_index = cpi_df.index[:-2]

raw_df = pd.read_excel(path_to_income
#                         ,encoding = "utf-8"
                       # ,error_bad_lines='ignore'
                       )

income_df = (raw_df.loc[(raw_df['Description']=='Per capita personal income (dollars) 2/') &
           (raw_df['GeoName'].str.contains(','))
          ].drop(columns=['GeoFIPS','Region','TableName','LineCode','IndustryClassification','Description','Unit'])
            .set_index('GeoName')
             .replace({'(NA)':np.nan})
             .astype(float)
             [income_index]
             
            )
income_df.index =  [i.replace('(Independent City)','city') for i in income_df.index]
# income_df = income_df.rename(index={'District of Columbia':'District of Columbia, DC'})
income_df.to_csv('csv/income.csv')
income_df

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
"Autauga, AL",15482.0,16417.0,17063.0,17700.0,18683.0,19350.0,20155.0,21076.0,22073.0,22948.0,...,32876.0,32603.0,33415.0,34325.0,35040.0,35464.0,36677.0,38591.0,39509.0,40484.0
"Baldwin, AL",15831.0,16954.0,18107.0,19106.0,20129.0,21153.0,22342.0,23434.0,24643.0,25373.0,...,35751.0,34698.0,36282.0,37804.0,38166.0,38212.0,39561.0,41412.0,43004.0,44079.0
"Barbour, AL",13490.0,14180.0,14943.0,15349.0,15930.0,15912.0,16109.0,16670.0,17651.0,19015.0,...,25678.0,26088.0,27834.0,28106.0,28145.0,30040.0,29941.0,31509.0,31750.0,33453.0
"Bibb, AL",12672.0,12896.0,13494.0,13936.0,14686.0,15352.0,15892.0,16375.0,16154.0,16475.0,...,24983.0,23789.0,25091.0,25973.0,27028.0,27420.0,28249.0,29048.0,28989.0,30022.0
"Blount, AL",13797.0,14269.0,14761.0,15205.0,15923.0,16537.0,17663.0,18109.0,19265.0,19721.0,...,27661.0,27381.0,27754.0,28368.0,29600.0,30188.0,31359.0,31987.0,32261.0,33707.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Sweetwater, WY",18111.0,18439.0,19487.0,20244.0,21577.0,22176.0,23424.0,24952.0,26366.0,27538.0,...,47177.0,40995.0,43231.0,46916.0,47511.0,47736.0,48978.0,49256.0,47555.0,49870.0
"Teton, WY",35525.0,35656.0,38354.0,42881.0,43988.0,48664.0,51107.0,54243.0,62147.0,68794.0,...,152558.0,125633.0,145017.0,151738.0,194277.0,178164.0,200007.0,204913.0,224366.0,233860.0
"Uinta, WY",15083.0,16215.0,16638.0,17522.0,17758.0,18800.0,19298.0,20631.0,21484.0,22504.0,...,40931.0,38479.0,38607.0,39840.0,40041.0,39091.0,39191.0,38931.0,37866.0,38797.0
"Washakie, WY",15944.0,17099.0,18175.0,19004.0,19613.0,20087.0,20994.0,22403.0,23538.0,25247.0,...,38757.0,36614.0,36464.0,41606.0,41120.0,41330.0,44218.0,44165.0,44204.0,45933.0


In [6]:
#CPI adjusted income 
base_year = 2010
income_df_adj = income_df.copy()
for year_ in income_df.columns:
    
    income_df_adj[year_] = (cpi_df[base_year] / cpi_df[year_]) * income_df[year_] 
    #income_df_adj[year_] = income_df[year_] / (1+((cpi_df[year_] - cpi_df[base_year]) / cpi_df[base_year]) )
    
income_df_adj.columns = ['Income_adj-'+str(col) for col in income_df_adj.columns]
income_df_adj     

,Income_adj-1990,Income_adj-1991,Income_adj-1992,Income_adj-1993,Income_adj-1994,Income_adj-1995,Income_adj-1996,Income_adj-1997,Income_adj-1998,Income_adj-1999,...,Income_adj-2008,Income_adj-2009,Income_adj-2010,Income_adj-2011,Income_adj-2012,Income_adj-2013,Income_adj-2014,Income_adj-2015,Income_adj-2016,Income_adj-2017
"Autauga, AL",25829.709197,26283.592893,26519.526215,26709.973702,27489.475358,27686.244094,28010.953983,28633.945520,29528.528147,30035.708812,...,33296.373279,33137.779348,33415.0,33274.675356,33279.102416,33195.559627,33782.947722,35503.778615,35895.513481,36014.111880
"Baldwin, AL",26411.970436,27143.329104,28142.123963,28831.681218,29617.066289,30266.001102,31050.396125,31837.534604,32966.589006,33209.693205,...,36208.134843,35267.143141,36282.0,36647.220020,36248.008641,35767.784922,36439.381488,38099.102900,39070.861367,39212.183518
"Barbour, AL",22506.315532,22702.159178,23224.595923,23162.225218,23438.812955,22767.106772,22387.916533,22647.934704,23612.923043,24887.964226,...,26006.335109,26515.915334,27834.0,27245.973068,26730.603239,28118.503586,27578.461645,28988.327858,28846.150321,29759.413218
"Bibb, AL",21141.588615,20646.477063,20972.542153,21029.954436,21608.437355,21965.851129,22086.334939,22247.146417,21610.286037,21563.460984,...,25302.448401,24179.205377,25091.0,25178.241603,25669.736875,25666.090824,26019.971377,26724.204120,26337.670918,26707.234138
"Blount, AL",23018.505218,22844.648047,22941.729266,22944.923737,23428.513414,23661.365302,24547.629879,24602.966380,25772.078773,25812.019064,...,28014.690998,27830.124109,27754.0,27499.956024,28112.483776,28257.036827,28884.572283,29428.088584,29310.414346,29985.368766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Sweetwater, WY",30215.854751,29520.811924,30286.937078,30548.966533,31747.599946,31729.723465,32554.134761,33899.896025,35271.561325,36043.374118,...,47780.234888,41667.431352,43231.0,45480.398224,45123.385698,44682.586125,45113.319343,45315.594814,43205.627669,44363.792102
"Teton, WY",59268.855394,57085.203642,59610.262466,64709.061149,64722.316653,69629.115381,71027.329458,73694.776374,83138.197742,90041.683457,...,154508.702842,127693.728578,145017.0,147094.907188,184513.817922,166767.812017,184225.155414,188520.271238,203845.523239,208039.230418
"Uinta, WY",25164.029441,25960.191189,25858.985944,26441.364927,26128.464561,26899.296588,26819.915156,28029.366579,28740.583460,29454.575174,...,41454.369591,39110.161995,38607.0,38620.919627,38028.782529,36590.560043,36098.576879,35816.579131,34402.781986,34513.375620
"Washakie, WY",26600.496282,27375.473891,28247.810406,28677.759336,28857.842969,28740.753753,29176.976826,30436.813508,31488.356613,33044.776903,...,39252.571455,37214.570839,36464.0,40332.881074,39053.558542,38686.343317,40728.914098,40631.867081,40161.109568,40861.481103


In [7]:
#read carfax data
path_to_carfax =path_to_csv+'SC-carfax\\processed\\MSRP.csv'

carfax = pd.read_csv(path_to_carfax,index_col=0) 
carfax['make'] = carfax['make'].str.upper()
carfax['model'] = carfax['model'].str.upper()
# carfax['year'] = carfax['year'].astype(str)

# carfax['make_key'] = carfax[['make','model','year']].agg('_'.join,axis=1)


carfax['Original_MSRP_mean'] =carfax[['Original_MSRP_low','Original_MSRP_high']].mean(axis=1)

#CPI adjusted income 
def cpi_adj(x):
    try:
        
        return round((cpi_df[base_year] / cpi_df[x['year']]) * x['Original_MSRP_mean'],2)
    except:
        return x['Original_MSRP_mean']


carfax['Original_MSRP_mean_adj'] = carfax.apply(cpi_adj
                                            ,axis=1)

carfax.head()

,make,model,year,Original_MSRP_low,Original_MSRP_high,Original_MSRP_mean,Original_MSRP_mean_adj
0,ACURA,CL,2001,28460.0,30810.0,29635.0,36488.37
1,ACURA,CL,2002,28530.0,32880.0,30705.0,37217.40
2,ACURA,CL,2003,28700.0,31050.0,29875.0,35404.47
3,ACURA,ILX,2013,25900.0,34400.0,30150.0,28221.47
4,ACURA,ILX,2014,26900.0,34600.0,30750.0,28323.63


In [8]:
carfax.to_csv(path_to_carfax)

In [9]:
# #Income

# path_to_income = path_to_database+'/US-Personal-Income/lapi1118_0.xlsx'
# income = (pd.read_excel(path_to_income
#                        #,sheet_name=1
#                        ,skiprows=3
# #                        ,index_col=0
#                        ,names = ['County',	'income-2015',	'income-2016'
#                                 ,'income-2017'
#                                 ,2017.1,2016.1,	2017.2,	2017.3]
#                       )
#          .iloc[:3216#,:4
#               ]
             
#          )

# df_list = [df.iloc[1:] for df in np.split(income,income[income.isnull().all(1)].index)][1:]

# df_list_res = []
# for i in range(len(df_list)):
#     #print(i)
#     df_list_res.append(df_list[i].assign(State = lambda x: x.iloc[0,0]).iloc[1:]
#                       )
# income_df = pd.concat(df_list_res,ignore_index=True)
# income_df['State-Code'] =income_df['State'].map(lambda x: state_codes.loc[x,'Postal Code'])
# # income_df['County'] = income_df['County'].map(str) +' County'

# income_df['County-State'] = income_df['County'].map(str) +', '+ income_df['State-Code'].map(str)
# income_df = income_df.set_index('County-State').iloc[:,:-2]
# income_df.sort_index()

In [10]:
#population data
path_to_pop = path_to_csv + 'US-Population\\cbsa-est2018-alldata.xlsx'
pop = pd.read_excel(path_to_pop,sheet_name=1#,index_col=0
                     )
pop['NAME'] =pop['NAME'].str.strip()
pop = (pop.set_index('NAME').rename({'CENSUS2010POP': 'CENSUS2010'},axis=1)
        .drop(['ESTIMATESBASE2010','POPESTIMATE2010'],axis=1)
        )
pop.columns = [col.replace('POPESTIMATE','CENSUS') for col in pop.columns]

pop.index = [', '.join([x.strip() for x in ind.replace(' County','').replace('/city','').replace('City','city').split(',')]
                            ) 
                 for ind in pop.index]


pop = pop.loc[~pop.index.duplicated(keep='first')]
pop.to_csv('csv/pop.csv')
pop.sort_index()

,CENSUS2010,CENSUS2011,CENSUS2012,CENSUS2013,CENSUS2014,CENSUS2015,CENSUS2016,CENSUS2017,CENSUS2018
"Abbeville, SC",25417,25060,25000,24880,24775,24775,24633,24560,24541
"Aberdeen, SD",40602,40962,41576,42155,42220,42404,42849,43235,43191
"Aberdeen, WA",72797,72357,71770,71046,70783,71027,71572,72553,73901
"Abilene, TX",165252,166616,167447,167472,168355,169704,170018,170516,171451
"Acadia Parish, LA",61773,61853,61991,62294,62664,62681,62793,62514,62190
...,...,...,...,...,...,...,...,...,...
"Yuba city, CA",166892,166977,167002,167659,168392,169471,170900,173213,174848
"Yuba, CA",72155,72492,72792,73114,73562,74083,75002,76775,78041
"Yuma, AZ",195751,202883,202589,202589,204283,205456,207520,209756,212128
"Zanesville, OH",86074,86236,85786,85640,85900,86174,85973,86148,86183


In [11]:
# #fibs codes
# path_to_c_codes = path_to_csv +'Us-zipcodes\\fips_codes_website.xls'
# county_codes = (pd.read_excel(path_to_c_codes)
# #                 .query('"FIPS Entity Code" ==0')
#                .assign( County= lambda x: x['GU Name'].str.strip()+ ', '+ x['State Abbreviation'].str.strip()
#                       )
#                 #.drop(columns=['Name','State'])
#                )
# county_codes['FIPS'] = county_codes['State FIPS Code'].map(lambda x: '{0:02d}'.format(x)) + county_codes['County FIPS Code'].map(lambda x: '{0:03d}'.format(x))                
# county_codes = county_codes.loc[county_codes['Entity Description']=='County',['FIPS','County']]

# county_codes.to_csv('csv/county_map.csv')
# county_codes.to_csv(path_to_csv +'Us-zipcodes\\fips_codes.csv')
# county_codes

In [12]:
#county_codes 
path_to_c_codes = path_to_csv+'/Us-zipcodes/US-FIBS-County-Codes.csv'
county_codes = (pd.read_csv(path_to_c_codes
                           #,index_col=0
                           )
               .assign( County= lambda x: x['Name'].str.strip()+ ', '+ x['State'].str.strip()
               ,FIPS = lambda x : x['FIPS'].map(lambda x: '{0:05d}'.format(x) )
                      )
                .drop(columns=['Name','State'])
               )
county_codes.to_csv('csv/county_map.csv')
county_codes

,FIPS,County
0,01001,"Autauga, AL"
1,01003,"Baldwin, AL"
2,01005,"Barbour, AL"
3,01007,"Bibb, AL"
4,01009,"Blount, AL"
...,...,...
3227,72151,"Yabucoa, PR"
3228,72153,"Yauco, PR"
3229,78010,"St. Croix, VI"
3230,78020,"St. John, VI"


In [13]:
county_codes.loc[ county_codes['County']=='Lake, IL']

,FIPS,County
639,17097,"Lake, IL"


In [14]:
#crime data 
path_to_cirme = path_to_csv+'US-Crime\\data\\'
path_to_crime_files = glob.glob(path_to_cirme+'*')

crime_df = pd.concat([(pd.read_csv(path_,sep='\t')
                       .assign(year = path_.split('-')[-1]
                                           .split('.')[0]
                              )
                      )
                      
                      for path_ in path_to_crime_files]
                      ,ignore_index=True)

print(crime_df.info()
      ,crime_df.head()
     ,(crime_df.reset_index()
       .groupby('year')['index'].count()
       .sort_values(ascending=False)
      )
     )

crime_df['FIPS'] = (crime_df['FIPS_ST'].map(lambda x: '{0:02d}'.format(x)) + crime_df['FIPS_CTY'].map(lambda x: '{0:03d}'.format(x)) )#.astype(int)                

crime_df_f_inp = crime_df[['FIPS'
                          ,'GRNDTOT' #Total number of insedents
                          # ,'CPOPARST' #TOT CNTY POPULATION-AGENCIES RPRT ARRSTS
                          ,'COVIND' #coverage indicatior if 0 then 0 in GRNDTOT is actually null
                          ,'year'
                         ]]


print('\n\n','Filtered columns:\n',crime_df_f_inp)


true_nulls = crime_df_f_inp.loc[lambda x : (x['COVIND'] == 0) & (x['GRNDTOT']==0)].index
crime_df_f_inp.loc[true_nulls,'GRNDTOT'] = np.nan


crime_df_f= crime_df_f_inp.set_index(['FIPS','year']).sort_index().unstack().copy()



crime_df_f.columns = crime_df_f.columns.map('_'.join)


print('\n\nUnstacked columns: \n' , crime_df_f)


#join fips and county columns

crime_df_filt = pd.merge(crime_df_f
                         ,county_codes
                         ,on = 'FIPS'
                         ,how='left'
                        )


crime_df_filt

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22243 entries, 0 to 22242
Data columns (total 57 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   STUDYNO   22243 non-null  int64  
 1   EDITION   22243 non-null  int64  
 2   PART      22243 non-null  int64  
 3   IDNO      22243 non-null  int64  
 4   FIPS_ST   22243 non-null  int64  
 5   FIPS_CTY  22243 non-null  int64  
 6   CPOPARST  22243 non-null  int64  
 7   AG_ARRST  22243 non-null  int64  
 8   JURFLAG   22243 non-null  int64  
 9   COVIND    22243 non-null  float64
 10  GRNDTOT   22243 non-null  int64  
 11  P1TOT     22243 non-null  int64  
 12  P1VLNT    22243 non-null  int64  
 13  P1PRPTY   22243 non-null  int64  
 14  MURDER    22243 non-null  int64  
 15  RAPE      22243 non-null  int64  
 16  ROBBERY   22243 non-null  int64  
 17  AGASSLT   22243 non-null  int64  
 18  BURGLRY   22243 non-null  int64  
 19  LARCENY   22243 non-null  int64  
 20  MVTHEFT   22243 non-null  in

,FIPS,GRNDTOT_2009,GRNDTOT_2010,GRNDTOT_2011,GRNDTOT_2012,GRNDTOT_2013,GRNDTOT_2014,GRNDTOT_2016,COVIND_2009,COVIND_2010,COVIND_2011,COVIND_2012,COVIND_2013,COVIND_2014,COVIND_2016,County
0,01001,1588.0,2710.0,NaN,825.0,NaN,974.0,2796.0,92.4317,98.4525,0.0000,0.0000,0.0000,0.0000,98.5035,"Autauga, AL"
1,01003,9990.0,8503.0,NaN,2195.0,NaN,4480.0,14765.0,51.2602,100.0000,0.0000,0.0000,0.0000,10.4765,96.2812,"Baldwin, AL"
2,01005,989.0,707.0,NaN,424.0,NaN,435.0,881.0,100.0000,98.9839,0.0000,0.0000,0.0000,32.4354,91.3039,"Barbour, AL"
3,01007,1118.0,780.0,NaN,241.0,NaN,246.0,751.0,100.0000,100.0000,0.0000,0.0000,0.0000,27.9504,90.4619,"Bibb, AL"
4,01009,1504.0,1979.0,NaN,254.0,NaN,804.0,2172.0,91.4786,99.2813,0.0000,0.0000,0.0000,7.3561,97.7122,"Blount, AL"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3180,56039,836.0,755.0,820.0,914.0,992.0,1031.0,1026.0,100.0000,100.0000,95.4146,45.0620,45.3187,45.8540,45.9667,"Teton, WY"
3181,56041,1304.0,1158.0,939.0,901.0,942.0,729.0,780.0,90.3882,89.9848,89.9850,89.9812,90.0745,100.0000,100.0000,"Uinta, WY"
3182,56043,382.0,306.0,286.0,272.0,248.0,220.0,93.0,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,"Washakie, WY"
3183,56045,251.0,212.0,211.0,248.0,231.0,240.0,243.0,87.2748,84.7392,84.7371,84.7455,84.7556,51.6374,84.5168,"Weston, WY"


In [15]:
true_nulls#.unique()

Int64Index([   80,    88,    92,    93,   325,   326,   327,   328,   329,
              330,
            ...
            19768, 19769, 20734, 20740, 20741, 20743, 20773, 20774, 20794,
            20798],
           dtype='int64', length=1370)

In [16]:
print('Thes FIPS codes don\'t match:' , crime_df_filt.loc[crime_df_filt['County'].isna(), 'FIPS'].unique().tolist())

dupl_fips  =crime_df_filt.set_index('FIPS').duplicated().loc[lambda x: x== True].index.tolist()
print('\nDuplicated values:', dupl_fips)
crime_df_filt.loc[lambda x: x['FIPS'].isin(dupl_fips)]

Thes FIPS codes don't match: ['01777', '02105', '02158', '02195', '02198', '02230', '02275', '02777', '02999', '04777', '06777', '08014', '08777', '09777', '09999', '12086', '12777', '15005', '16777', '20777', '22777', '23777', '25777', '26777', '27777', '28777', '30777', '31777', '32777', '34999', '35777', '36777', '37777', '38777', '40777', '41777', '44777', '46102', '46777', '48777', '49777', '50999', '53777', '55777', '56777']

Duplicated values: ['12777', '15005', '44777']


,FIPS,GRNDTOT_2009,GRNDTOT_2010,GRNDTOT_2011,GRNDTOT_2012,GRNDTOT_2013,GRNDTOT_2014,GRNDTOT_2016,COVIND_2009,COVIND_2010,COVIND_2011,COVIND_2012,COVIND_2013,COVIND_2014,COVIND_2016,County
398,12777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,NaN
560,15005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2347,44777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,NaN


In [17]:
crime_df_filt_dropnna = (crime_df_filt.drop_duplicates(subset=['FIPS'])
                        .dropna(subset=['County']).drop(['FIPS'],axis=1)
                        .set_index('County')
                        )

crime_df_filt_dropnna.loc['Autauga, AL']

GRNDTOT_2009    1588.0000
GRNDTOT_2010    2710.0000
GRNDTOT_2011          NaN
GRNDTOT_2012     825.0000
GRNDTOT_2013          NaN
GRNDTOT_2014     974.0000
GRNDTOT_2016    2796.0000
COVIND_2009       92.4317
COVIND_2010       98.4525
COVIND_2011        0.0000
COVIND_2012        0.0000
COVIND_2013        0.0000
COVIND_2014        0.0000
COVIND_2016       98.5035
Name: Autauga, AL, dtype: float64

In [18]:
crime_df_filt_dropnna

,GRNDTOT_2009,GRNDTOT_2010,GRNDTOT_2011,GRNDTOT_2012,GRNDTOT_2013,GRNDTOT_2014,GRNDTOT_2016,COVIND_2009,COVIND_2010,COVIND_2011,COVIND_2012,COVIND_2013,COVIND_2014,COVIND_2016
County,,,,,,,,,,,,,,
"Autauga, AL",1588.0,2710.0,NaN,825.0,NaN,974.0,2796.0,92.4317,98.4525,0.0000,0.0000,0.0000,0.0000,98.5035
"Baldwin, AL",9990.0,8503.0,NaN,2195.0,NaN,4480.0,14765.0,51.2602,100.0000,0.0000,0.0000,0.0000,10.4765,96.2812
"Barbour, AL",989.0,707.0,NaN,424.0,NaN,435.0,881.0,100.0000,98.9839,0.0000,0.0000,0.0000,32.4354,91.3039
"Bibb, AL",1118.0,780.0,NaN,241.0,NaN,246.0,751.0,100.0000,100.0000,0.0000,0.0000,0.0000,27.9504,90.4619
"Blount, AL",1504.0,1979.0,NaN,254.0,NaN,804.0,2172.0,91.4786,99.2813,0.0000,0.0000,0.0000,7.3561,97.7122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Sweetwater, WY",3459.0,2927.0,2650.0,1966.0,1757.0,1625.0,1966.0,100.0000,100.0000,100.0000,95.6181,100.0000,95.5159,100.0000
"Teton, WY",836.0,755.0,820.0,914.0,992.0,1031.0,1026.0,100.0000,100.0000,95.4146,45.0620,45.3187,45.8540,45.9667
"Uinta, WY",1304.0,1158.0,939.0,901.0,942.0,729.0,780.0,90.3882,89.9848,89.9850,89.9812,90.0745,100.0000,100.0000


In [19]:
cols_to_save =[col for col in crime_df_filt_dropnna if 'GRNDTOT' in col]
crime_df_filt_dropnna = crime_df_filt_dropnna[cols_to_save].copy()

crime_df_filt_dropnna = crime_df_filt_dropnna.rename(index={'Washington, DC':'District of Columbia, DC'})
crime_df_filt_dropnna.index =  [i.replace('City','city') for i in crime_df_filt_dropnna.index]



crime_df_filt_dropnna.to_csv('csv/crime.csv')

In [20]:
# joint_full = unempdf.join(income_df).join(pop)

joint_full = unempdf.join(income_df_adj).join(pop).join(crime_df_filt_dropnna)

In [21]:
joint_full.columns

Index(['Unemp-Pct-2014', 'Unemp-Pct-2015', 'Unemp-Pct-2016', 'Unemp-Pct-2017',
       'Unemp-Pct-2018', 'Unemp-Pct-2013', 'Unemp-Pct-2012', 'Unemp-Pct-2011',
       'Unemp-Pct-2010', 'Income_adj-1990', 'Income_adj-1991',
       'Income_adj-1992', 'Income_adj-1993', 'Income_adj-1994',
       'Income_adj-1995', 'Income_adj-1996', 'Income_adj-1997',
       'Income_adj-1998', 'Income_adj-1999', 'Income_adj-2000',
       'Income_adj-2001', 'Income_adj-2002', 'Income_adj-2003',
       'Income_adj-2004', 'Income_adj-2005', 'Income_adj-2006',
       'Income_adj-2007', 'Income_adj-2008', 'Income_adj-2009',
       'Income_adj-2010', 'Income_adj-2011', 'Income_adj-2012',
       'Income_adj-2013', 'Income_adj-2014', 'Income_adj-2015',
       'Income_adj-2016', 'Income_adj-2017', 'CENSUS2010', 'CENSUS2011',
       'CENSUS2012', 'CENSUS2013', 'CENSUS2014', 'CENSUS2015', 'CENSUS2016',
       'CENSUS2017', 'CENSUS2018', 'GRNDTOT_2009', 'GRNDTOT_2010',
       'GRNDTOT_2011', 'GRNDTOT_2012', 'GRNDTOT_20

In [22]:
joint_full_nn = joint_full.loc[~joint_full.isnull().any(1)]#.drop(columns=['County'])

In [23]:
print(joint_full.loc['Los Angeles, CA'].to_frame().T,'\n',
joint_full.loc['Philadelphia, PA'].to_frame().T,'\n',
joint_full.loc['Denver, CO'].to_frame().T,'\n',
joint_full.loc['District of Columbia, DC'].to_frame().T,'\n',

joint_full.loc['Hampton city, VA'].to_frame().T,'\n',

joint_full.loc['Honolulu, HI'].to_frame().T,'\n',
joint_full.loc['Baltimore city, MD'].to_frame().T



)

Unemp-Pct-2014  Unemp-Pct-2015  Unemp-Pct-2016  \
Los Angeles, CA             8.3             6.6             5.3   

                 Unemp-Pct-2017  Unemp-Pct-2018  Unemp-Pct-2013  \
Los Angeles, CA             4.8             4.7             9.8   

                 Unemp-Pct-2012  Unemp-Pct-2011  Unemp-Pct-2010  \
Los Angeles, CA            10.9            12.2            12.5   

                 Income_adj-1990  ...  CENSUS2016  CENSUS2017  CENSUS2018  \
Los Angeles, CA     35886.645448  ...  10120540.0  10118759.0  10105518.0   

                 GRNDTOT_2009  GRNDTOT_2010  GRNDTOT_2011  GRNDTOT_2012  \
Los Angeles, CA      376777.0      363182.0      340483.0      326586.0   

                 GRNDTOT_2013  GRNDTOT_2014  GRNDTOT_2016  
Los Angeles, CA      302215.0      290955.0      254215.0  

[1 rows x 53 columns] 
                   Unemp-Pct-2014  Unemp-Pct-2015  Unemp-Pct-2016  \
Philadelphia, PA             8.1             7.1             6.8   

                  Unemp-

In [24]:
path_to_processed_csv =path_to_csv+'SC-working-folder\\'

joint_full.reset_index().to_csv(path_to_processed_csv+'demography.csv',index=False)